In [1]:
import pandas as pd

In [2]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [3]:
ARQUIVO_TODAS_DEMONSTRACOES = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS_GERAIS['arquivo_final_todas_demonstracoes']

In [4]:
ARQUIVO_TODAS_DEMONSTRACOES

'D:/tccdata/todas_demonstracoes_concatenadas.csv'

In [5]:
df_demonstracoes_plano_contas = pd.read_csv(ARQUIVO_TODAS_DEMONSTRACOES,encoding="ISO-8859-1")

In [6]:
df_idss = pd.read_csv("../entrada/idss_alto_nivel.csv",encoding="ISO-8859-1")

In [7]:
df_demonstracoes = df_demonstracoes_plano_contas.copy()

In [8]:
len(df_demonstracoes_plano_contas)

9601

In [9]:
df_demonstracoes.columns

Index(['REG_ANS', 'ANO', 'SEMESTRE', 'DESPESA DE COMERCIALIZAÇÃO DIFERIDA',
       'ATIVO NÃO CIRCULANTE INTANGÍVEL', 'DESPESAS ANTECIPADAS',
       'CRÉDITO TRIBUTÁRIO', 'PARTICIPAÇÕES OPS',
       'PARTICIPAÇÕES INSTITUIÇÕES REGULADAS', 'PARTICIPAÇÕES COOPERATIVAS',
       'APLICAÇÕES GARANTIDORAS', 'PPSC', 'RECEITAS FINANCEIRAS',
       'DESPESAS FINANCEIRAS', 'OUTRAS RECEITAS OPERACIONAIS',
       'OUTRAS DESPESAS OPERACIONAIS', 'DESPESA', 'RECEITA',
       'TRIBUTOS DIRETOS', 'IMPOSTO SOBRE LUCRO', 'PEONA_SUS', 'PEONA_OUTROS',
       'PESL_OUTROS', 'PESL_SUS', 'REMISSAO', 'PPCNG', 'PROVISOES_TECNICAS',
       'DESPESA_ADM', 'DESPESA_COM', 'VARIACAO_PEONA',
       'EVENTOS_INDENIZAVEIS_POS', 'EVENTOS_INDENIZAVEIS_PRE',
       'EVENTOS_INDENIZAVEIS_LIQUIDOS', 'CORRESP_CEDIDA_POS',
       'CORRESP_CEDIDA_PRE', 'CONTRAPRESTACAO_POS', 'CONTRAPRESTACAO_PRE',
       'RECEITA_COM_ASSISTENCIA_A_SAUDE', 'PL', 'PASSIVO_NAO_CIRCULANTE',
       'PASSIVO_CIRCULANTE', 'PASSIVO', 'CREDITO_OPER_PS

In [10]:
df_demons = df_demonstracoes.merge(df_idss,left_on=['REG_ANS','ANO'],right_on=['Registro_ANS','ano'])

In [11]:
df_demons.columns

Index(['REG_ANS', 'ANO', 'SEMESTRE', 'DESPESA DE COMERCIALIZAÇÃO DIFERIDA',
       'ATIVO NÃO CIRCULANTE INTANGÍVEL', 'DESPESAS ANTECIPADAS',
       'CRÉDITO TRIBUTÁRIO', 'PARTICIPAÇÕES OPS',
       'PARTICIPAÇÕES INSTITUIÇÕES REGULADAS', 'PARTICIPAÇÕES COOPERATIVAS',
       'APLICAÇÕES GARANTIDORAS', 'PPSC', 'RECEITAS FINANCEIRAS',
       'DESPESAS FINANCEIRAS', 'OUTRAS RECEITAS OPERACIONAIS',
       'OUTRAS DESPESAS OPERACIONAIS', 'DESPESA', 'RECEITA',
       'TRIBUTOS DIRETOS', 'IMPOSTO SOBRE LUCRO', 'PEONA_SUS', 'PEONA_OUTROS',
       'PESL_OUTROS', 'PESL_SUS', 'REMISSAO', 'PPCNG', 'PROVISOES_TECNICAS',
       'DESPESA_ADM', 'DESPESA_COM', 'VARIACAO_PEONA',
       'EVENTOS_INDENIZAVEIS_POS', 'EVENTOS_INDENIZAVEIS_PRE',
       'EVENTOS_INDENIZAVEIS_LIQUIDOS', 'CORRESP_CEDIDA_POS',
       'CORRESP_CEDIDA_PRE', 'CONTRAPRESTACAO_POS', 'CONTRAPRESTACAO_PRE',
       'RECEITA_COM_ASSISTENCIA_A_SAUDE', 'PL', 'PASSIVO_NAO_CIRCULANTE',
       'PASSIVO_CIRCULANTE', 'PASSIVO', 'CREDITO_OPER_PS

#### Margem de lucro líquida (MLL)

In [12]:

df_demons['RESULTADO_LIQUIDO'] = df_demons['RECEITA']-df_demons['DESPESA']-df_demons['IMPOSTO SOBRE LUCRO']
df_demons['CORRESPONSABILIDADE_CEDIDA_EFETIVA'] = df_demons['CORRESP_CEDIDA_PRE'].abs() + df_demons['CORRESP_CEDIDA_POS'].abs()
df_demons['CONTRAPRESTACOES_EFETIVAS'] = df_demons['RECEITA_COM_ASSISTENCIA_A_SAUDE'] + \
                                    df_demons['CORRESPONSABILIDADE_CEDIDA_EFETIVA'] +\
                                    df_demons['TRIBUTOS DIRETOS']
df_demons['MLL'] = df_demons['RESULTADO_LIQUIDO']/df_demons['CONTRAPRESTACOES_EFETIVAS']


#### Retorno sobre o Patrimônio Líquido (ROE)

In [13]:
df_demons["ROE"] = df_demons['RESULTADO_LIQUIDO']/df_demons['PL']


#### Retorno sobre o Ativo (ROA)

In [14]:
df_demons["ROA"] = df_demons['RESULTADO_LIQUIDO']/df_demons['ATIVO']

#### Percentual de Despesas Assistenciais em relação às Receitas de Contraprestações (DM)

In [15]:
df_demons['Eventos indenizáveis líquidos'] = (df_demons['DESPESA']+ \
                                df_demons['CORRESPONSABILIDADE_CEDIDA_EFETIVA'])
df_demons['DM'] = df_demons['Eventos indenizáveis líquidos']/df_demons['CONTRAPRESTACOES_EFETIVAS']


#### Percentual de Despesas Administrativas em relação às Receitas de Contraprestações (DA)

In [16]:
df_demons['DA'] = df_demons['DESPESA_ADM']/df_demons['CONTRAPRESTACOES_EFETIVAS']

#### Percentual de Despesa Comercial em relação à Receita de Contraprestações (DC)

In [17]:
df_demons['DC'] = df_demons['DESPESA_COM']/df_demons['CONTRAPRESTACOES_EFETIVAS']

### Percentual de Despesas Operacionais em relação às Receitas Operacionais (DOP)

In [18]:
df_demons['DOP'] = (df_demons['Eventos indenizáveis líquidos']+df_demons['DESPESA_ADM']+ \
                    df_demons['DESPESA_COM'] + df_demons['OUTRAS DESPESAS OPERACIONAIS'])/ \
                    (df_demons['CONTRAPRESTACOES_EFETIVAS'] + df_demons['OUTRAS RECEITAS OPERACIONAIS'])

#### Índice de Resultado Financeiro (IRF)

In [19]:
df_demons['RESULTADO_FINANCEIRO_LIQUIDO'] = df_demons['RECEITAS FINANCEIRAS'] - df_demons['DESPESAS FINANCEIRAS']
df_demons['IRF'] = df_demons['RESULTADO_FINANCEIRO_LIQUIDO']/df_demons['CONTRAPRESTACOES_EFETIVAS']

#### Liquidez Corrente (LC)

In [20]:
df_demons['LC'] = df_demons['ATIVO_CIRCULANTE']/df_demons['PASSIVO_CIRCULANTE']

#### Capital de terceiros sobre o Capital próprio (CT/CP)

In [21]:
df_demons['CT/CP'] = (df_demons['PASSIVO_CIRCULANTE'] + df_demons['PASSIVO_NAO_CIRCULANTE'])/ \
                        df_demons['PL']


### Prazo Médio de Recebimento de Contraprestações (PMRC)

In [22]:
df_demons['PMRC'] = ((df_demons['CREDITO_OPER_PS'] + df_demons['PPSC'].abs())/ \
                     df_demons['CONTRAPRESTACOES_EFETIVAS'])*360

### Prazo Médio de Pagamento de Eventos (PMPE)

In [23]:
df_demons['PROVISAO_EVENTOS_A_LIQUIDAR'] = df_demons['PESL_OUTROS']+df_demons['PEONA_SUS']
df_demons['PMPE'] = (df_demons['PROVISAO_EVENTOS_A_LIQUIDAR']/df_demons['Eventos indenizáveis líquidos'])*360

#### Suficiência de lastro das provisões técnicas

In [24]:
df_demons['PROVISOES_TECNICAS_EXIGIDAS'] = df_demons['REMISSAO'] + df_demons['PESL_SUS']+ \
                                        df_demons['PESL_OUTROS'] + df_demons['PEONA_OUTROS']
df_demons['Lastro P T'] = df_demons['APLICAÇÕES GARANTIDORAS']/df_demons['PROVISOES_TECNICAS_EXIGIDAS']

In [25]:
df_demons.columns

Index(['REG_ANS', 'ANO', 'SEMESTRE', 'DESPESA DE COMERCIALIZAÇÃO DIFERIDA',
       'ATIVO NÃO CIRCULANTE INTANGÍVEL', 'DESPESAS ANTECIPADAS',
       'CRÉDITO TRIBUTÁRIO', 'PARTICIPAÇÕES OPS',
       'PARTICIPAÇÕES INSTITUIÇÕES REGULADAS', 'PARTICIPAÇÕES COOPERATIVAS',
       'APLICAÇÕES GARANTIDORAS', 'PPSC', 'RECEITAS FINANCEIRAS',
       'DESPESAS FINANCEIRAS', 'OUTRAS RECEITAS OPERACIONAIS',
       'OUTRAS DESPESAS OPERACIONAIS', 'DESPESA', 'RECEITA',
       'TRIBUTOS DIRETOS', 'IMPOSTO SOBRE LUCRO', 'PEONA_SUS', 'PEONA_OUTROS',
       'PESL_OUTROS', 'PESL_SUS', 'REMISSAO', 'PPCNG', 'PROVISOES_TECNICAS',
       'DESPESA_ADM', 'DESPESA_COM', 'VARIACAO_PEONA',
       'EVENTOS_INDENIZAVEIS_POS', 'EVENTOS_INDENIZAVEIS_PRE',
       'EVENTOS_INDENIZAVEIS_LIQUIDOS', 'CORRESP_CEDIDA_POS',
       'CORRESP_CEDIDA_PRE', 'CONTRAPRESTACAO_POS', 'CONTRAPRESTACAO_PRE',
       'RECEITA_COM_ASSISTENCIA_A_SAUDE', 'PL', 'PASSIVO_NAO_CIRCULANTE',
       'PASSIVO_CIRCULANTE', 'PASSIVO', 'CREDITO_OPER_PS

In [26]:
df_indicadores = df_demons.groupby(['ANO','SEMESTRE','porte'],as_index=False).mean()

In [27]:
df_indicadores = df_indicadores[[
    'ANO','SEMESTRE','porte','MLL','ROE','ROA','DM','DA','DC','DOP','IRF','LC','CT/CP','PMRC','PMPE','Lastro P T'
]]

In [29]:
df_indicadores.to_csv('../entrada/indicadores_cooperativas.csv',index=False,encoding="ISO-8859-1",float_format='%.2f')